In [1]:
import sqlalchemy 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect
from sqlalchemy import Integer, Column, Float, String
import numpy as np

In [2]:
import pandas as pd
import os
import csv
import requests
#from config import api_key
from pprint import pprint
import matplotlib.pyplot as plt


In [3]:
input_file= os.path.join("TMDB_OMDB-3.csv")
df_new=pd.read_csv('TMDB_OMDB-3.csv', encoding = 'utf8')
df_new.head()


,show_id,type,title,director,cast,country,date_added,Netflix release year,rating,duration,listed_in,description,IMDB rating,Rotten Tomatoes rating,Award,Released Date,Budget,Movie ID,Revenue
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,3,36,NaN,2-Aug-19,0,601131.0,1442504
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,5,0,NaN,NaN,0,415722.0,0
2,80163890,TV Show,Apaches,NaN,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",Spain,8-Sep-17,2016,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Spanis...",A young journalist is forced into a life of cr...,5,31,2 nominations.,14-Aug-13,0,164337.0,0
3,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",6,29,6 nominations.,17-Oct-14,7000000,262543.0,0
4,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden",8-Sep-17,2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,5,12,NaN,21-Aug-15,0,262338.0,0


In [4]:
engine = create_engine("postgres://pgadmin@pg-srv-001:ucN-xZRL3NsaBjvG2tcw1gPcsNeS5Xfw@pg-srv-001.postgres.database.azure.com:5432/netflix",connect_args={'sslmode':'require'})

conn=engine.connect()
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['listed_in', 'title', 'players', 'pg_rating', 'country', 'player_title']

In [5]:
inspector = inspect(engine)
inspector.get_table_names()

['listed_in',
 'director',
 'title',
 'players',
 'pg_rating',
 'country',
 'player_title']

In [6]:
title_df = pd.DataFrame(df_new,columns=['show_id','show_type','title','date_added','Netflix release year','duration','description','IMDB rating','Rotten Tomatoes rating','Award','Released Date','Budget','Movie ID','Revenue'])

title_df = title_df.reset_index(drop=True)
title_df.head()

,show_id,show_type,title,date_added,Netflix release year,duration,description,IMDB rating,Rotten Tomatoes rating,Award,Released Date,Budget,Movie ID,Revenue
0,81145628,NaN,Norm of the North: King Sized Adventure,9-Sep-19,2019,90 min,Before planning an awesome wedding for his gra...,3,36,NaN,2-Aug-19,0,601131.0,1442504
1,80117401,NaN,Jandino: Whatever it Takes,9-Sep-16,2016,94 min,Jandino Asporaat riffs on the challenges of ra...,5,0,NaN,NaN,0,415722.0,0
2,80163890,NaN,Apaches,8-Sep-17,2016,1 Season,A young journalist is forced into a life of cr...,5,31,2 nominations.,14-Aug-13,0,164337.0,0
3,70304989,NaN,Automata,8-Sep-17,2014,110 min,"In a dystopian future, an insurance adjuster f...",6,29,6 nominations.,17-Oct-14,7000000,262543.0,0
4,70304990,NaN,Good People,8-Sep-17,2014,90 min,A struggling couple can't believe their luck w...,5,12,NaN,21-Aug-15,0,262338.0,0


In [7]:
title_df["description"] = title_df["description"].str.replace("'","''")
title_df["title"] = title_df["title"].str.replace("'","''")

In [9]:
show_id_list = list(title_df['show_id'])
show_type_list = list(title_df['show_type'])
title_list = list(title_df['title'])
date_added_list = list(title_df['date_added'])
Netflix_release_year_list = list(title_df['Netflix release year'])
duration_list = list(title_df['duration'])
description_list = list(title_df['description'])
IMDB_rating_list = list(title_df['IMDB rating'])
Rotten_Tomatoes_rating_list = list(title_df['Rotten Tomatoes rating'])
Award_list = list(title_df['Award'])
Released_Date_list = list(title_df['Released Date'])
Budget_list = list(title_df['Budget'])
Movie_ID_list = list(title_df['Movie ID'])
Revenue_list = list(title_df['Revenue'])


for i in range (len (show_id_list)):
    statement = f"INSERT INTO title VALUES ({show_id_list[i]},'{show_type_list[i]}','{title_list[i]}','{date_added_list[i]}','{Netflix_release_year_list[i]}','{duration_list[i]}','{description_list[i]}','{IMDB_rating_list[i]}',{Rotten_Tomatoes_rating_list[i]},'{Award_list[i]}','{Released_Date_list[i]}',{Budget_list[i]},{Revenue_list[i]},'{Movie_ID_list[i]}')"
    #print (statement)
    conn.execute(statement)
    #      print (show_id_list,show_type_list,title_list,date_added_list,Netflix_release_year_list,duration_list,description_list,IMDB_rating_list,Rotten_Tomatoes_rating_list,Award_list,Released_Date_list,Movie_ID_list,Revenue_list)


In [81]:
Director_df = pd.DataFrame(df_new,columns=['director'])
#Director_df = Director_df.set_index("director")
Director_df = Director_df.reset_index(drop=True)
Director_df.head()

,director
0,"Richard Finn, Tim Maltby"
1,NaN
2,NaN
3,Gabe Ibáñez
4,Henrik Ruben Genz


In [82]:
Director_df.director = Director_df.director.str.split(',')
Director_df = Director_df.explode('director').reset_index(drop=True)
Director_df.drop_duplicates(subset=['director'], inplace=True)
Director_df.dropna(subset=['director'],inplace=True)
Director_df['director'] = Director_df['director'].str.strip()
Director_df

,director
0,Richard Finn
1,Tim Maltby
4,Gabe Ibáñez
5,Henrik Ruben Genz
6,José Miguel Contreras
...,...
3734,Kike Maíllo
3735,G.J. Echternkamp
3736,Zatella Beatty
3766,Glen Winter


In [83]:
Director_df["director"] = Director_df["director"].str.replace("'","''")

In [84]:
for director in Director_df['director']: 
    statement = f"INSERT INTO Director (director_name) VALUES ('{director}')"
    conn.execute(statement)

In [11]:
Listed_in_df = pd.DataFrame(df_new,columns=['listed_in'])
Listed_in_df = Listed_in_df.reset_index(drop=True)
Listed_in_df.head()

,listed_in
0,"Children & Family Movies, Comedies"
1,Stand-Up Comedy
2,"Crime TV Shows, International TV Shows, Spanis..."
3,"International Movies, Sci-Fi & Fantasy, Thrillers"
4,"Action & Adventure, Thrillers"


In [12]:
Listed_in_df.listed_in = Listed_in_df.listed_in.str.split(',')
Listed_in_df = Listed_in_df.explode('listed_in').reset_index(drop=True)
Listed_in_df.drop_duplicates(subset=['listed_in'], inplace=True)
Listed_in_df.dropna(subset=['listed_in'],inplace=True)
Listed_in_df

,listed_in
0,Children & Family Movies
1,Comedies
2,Stand-Up Comedy
3,Crime TV Shows
4,International TV Shows
...,...
3180,Romantic Movies
5223,Kids' TV
7115,TV Sci-Fi & Fantasy
7134,Classic & Cult TV


In [13]:
Listed_in_df['listed_in'] = Listed_in_df['listed_in'].str.strip()
Listed_in_df.drop_duplicates (subset = ['listed_in'], inplace = True) 
Listed_in_df.reset_index(inplace = True )
Listed_in_df

,index,listed_in
0,0,Children & Family Movies
1,1,Comedies
2,2,Stand-Up Comedy
3,3,Crime TV Shows
4,4,International TV Shows
5,5,Spanish-Language TV Shows
6,6,International Movies
7,7,Sci-Fi & Fantasy
8,8,Thrillers
9,9,Action & Adventure


In [14]:
Listed_in_df["listed_in"] = Listed_in_df["listed_in"].str.replace("'","''")

In [39]:
i =1
for cols in Listed_in_df['listed_in']: 
    statement = f"INSERT INTO listed_in VALUES ({i},'{cols}')"
    conn.execute(statement)
    i=i+1

In [15]:
cast_df = pd.DataFrame(df_new,columns=['cast'])
cast_df = cast_df.reset_index(drop=True)
cast_df.head()

,cast
0,"Alan Marriott, Andrew Toth, Brian Dobson, Cole..."
1,Jandino Asporaat
2,"Alberto Ammann, Eloy Azorín, Verónica Echegui,..."
3,"Antonio Banderas, Dylan McDermott, Melanie Gri..."
4,"James Franco, Kate Hudson, Tom Wilkinson, Omar..."


In [16]:
cast_df.cast =cast_df.cast.str.split(',')
cast_df = cast_df.explode('cast').reset_index(drop=True)
cast_df.drop_duplicates(subset=['cast'], inplace=True)
cast_df.dropna(subset=['cast'],inplace=True)
cast_df

,cast
0,Alan Marriott
1,Andrew Toth
2,Brian Dobson
3,Cole Howard
4,Jennifer Cameron
...,...
24506,Craig Behenna
24507,Charlotte Hamlyn
24508,Stavroula Mountzouris
24509,Aletheia Burney


In [17]:
cast_df["cast"] = cast_df["cast"].str.replace("'","''")

In [18]:
cast_df["cast"] = cast_df["cast"].str.strip()
cast_df.drop_duplicates (subset = ['cast'], inplace = True) 
cast_df.reset_index(inplace = True )
cast_df

,index,cast
0,0,Alan Marriott
1,1,Andrew Toth
2,2,Brian Dobson
3,3,Cole Howard
4,4,Jennifer Cameron
...,...,...
16290,24506,Craig Behenna
16291,24507,Charlotte Hamlyn
16292,24508,Stavroula Mountzouris
16293,24509,Aletheia Burney


In [56]:
i =1
for cols in cast_df['cast']: 
    statement = f"INSERT INTO players VALUES ({i},'{cols}')"
    conn.execute(statement)
    i=i+1


In [19]:
pg_rating_df = pd.DataFrame(df_new,columns=['rating'])
pg_rating_df = pg_rating_df.reset_index(drop=True)
pg_rating_df.head()

,rating
0,TV-PG
1,TV-MA
2,TV-MA
3,R
4,R


In [20]:
pg_rating_df.drop_duplicates(subset=['rating'], inplace=True)
pg_rating_df.dropna(subset=['rating'],inplace=True)
pg_rating_df

,rating
0,TV-PG
1,TV-MA
3,R
7,TV-14
10,PG-13
12,NR
28,PG
42,TV-Y7
43,TV-G
86,TV-Y


In [59]:
for cols in pg_rating_df['rating']:
    statement = f"INSERT INTO pg_rating (pg_rating_name) VALUES ('{cols}')"
    conn.execute(statement)

In [21]:
Country_df = pd.DataFrame(df_new,columns=['country'])
Country_df = Country_df.reset_index(drop=True)
Country_df.head(50)

,country
0,"United States, India, South Korea, China"
1,United Kingdom
2,Spain
3,"Bulgaria, United States, Spain, Canada"
4,"United States, United Kingdom, Denmark, Sweden"
5,NaN
6,"Netherlands, Belgium, United Kingdom, United S..."
7,United States
8,"France, Belgium"
9,"United States, Uruguay"


In [22]:
input_file= os.path.join("countries.csv")
Country_df=pd.read_csv('countries.csv', encoding = 'utf8')
Country_df.tail()

,country,latitude,longitude,name
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia
244,ZW,-19.015438,29.154857,Zimbabwe


In [23]:
Country_df["name"] = Country_df["name"].str.replace("'","''")

In [24]:
fill_values = {'country':"N/A"}
Country_df.fillna(value=fill_values, inplace=True)

print(Country_df.loc[Country_df['name']== 'Namibia'])



    country  latitude  longitude     name
156     N/A -22.95764   18.49041  Namibia


In [80]:
country_list = list(Country_df['country'])
lat_list = list(Country_df['latitude'])
long_list = list(Country_df['longitude'])
name_list= list(Country_df['name'])


for i in range (len (country_list)):

    statement = f"INSERT INTO country VALUES  ({i},'{name_list[i]}','{country_list[i]}','{round(long_list[i],6)}','{round(lat_list[i],6)}')"
    conn.execute(statement)

In [25]:
netflix_titles_Cast = pd.DataFrame ({"Titles": df_new["show_id"],
                                          "Cast": df_new["cast"]}) 
netflix_titles_Cast.Cast = netflix_titles_Cast.Cast.str.split(', ')
netflix_titles_Cast = netflix_titles_Cast.explode('Cast').reset_index(drop=True)
netflix_titles_Cast.dropna(subset=['Cast'], inplace=True)

In [26]:
netflix_titles_Cast["Cast"] = netflix_titles_Cast["Cast"].str.replace("'","''")

In [27]:
netflix_titles_Cast.drop_duplicates (subset = ['Titles','Cast'], inplace = True) 
netflix_titles_Cast.head()

,Titles,Cast
0,81145628,Alan Marriott
1,81145628,Andrew Toth
2,81145628,Brian Dobson
3,81145628,Cole Howard
4,81145628,Jennifer Cameron


In [32]:
player_name_list = list(netflix_titles_Cast['Cast'])
show_id_list = list(netflix_titles_Cast['Titles'])
player_tile_statements = []
for i in range(len(player_name_list)):
    statement_player_id = engine.execute(f"select player_id from players where player_name = '{player_name_list[i]}'").fetchall()
    player_tile_statements.append (f"INSERT INTO player_title (player_id, show_id) values ({statement_player_id[0][0]},{show_id_list[i]})")
    print (i, " - ", player_tile_statements[i])

0  -  INSERT INTO player_title (player_id, show_id) values (1,81145628)
1  -  INSERT INTO player_title (player_id, show_id) values (2,81145628)
2  -  INSERT INTO player_title (player_id, show_id) values (3,81145628)
3  -  INSERT INTO player_title (player_id, show_id) values (4,81145628)
4  -  INSERT INTO player_title (player_id, show_id) values (5,81145628)
5  -  INSERT INTO player_title (player_id, show_id) values (6,81145628)
6  -  INSERT INTO player_title (player_id, show_id) values (7,81145628)
7  -  INSERT INTO player_title (player_id, show_id) values (8,81145628)
8  -  INSERT INTO player_title (player_id, show_id) values (9,81145628)
9  -  INSERT INTO player_title (player_id, show_id) values (10,81145628)
10  -  INSERT INTO player_title (player_id, show_id) values (11,80117401)
11  -  INSERT INTO player_title (player_id, show_id) values (12,80163890)
12  -  INSERT INTO player_title (player_id, show_id) values (13,80163890)
13  -  INSERT INTO player_title (player_id, show_id) valu

111  -  INSERT INTO player_title (player_id, show_id) values (111,81154455)
112  -  INSERT INTO player_title (player_id, show_id) values (112,81154455)
113  -  INSERT INTO player_title (player_id, show_id) values (113,81154455)
114  -  INSERT INTO player_title (player_id, show_id) values (114,81154455)
115  -  INSERT INTO player_title (player_id, show_id) values (115,81154455)
116  -  INSERT INTO player_title (player_id, show_id) values (116,81154455)
117  -  INSERT INTO player_title (player_id, show_id) values (117,81052275)
118  -  INSERT INTO player_title (player_id, show_id) values (118,81052275)
119  -  INSERT INTO player_title (player_id, show_id) values (119,81052275)
120  -  INSERT INTO player_title (player_id, show_id) values (120,81052275)
121  -  INSERT INTO player_title (player_id, show_id) values (121,81052275)
122  -  INSERT INTO player_title (player_id, show_id) values (122,81052275)
123  -  INSERT INTO player_title (player_id, show_id) values (123,80162141)
124  -  INSE

220  -  INSERT INTO player_title (player_id, show_id) values (217,80113641)
221  -  INSERT INTO player_title (player_id, show_id) values (218,80113641)
222  -  INSERT INTO player_title (player_id, show_id) values (219,80113641)
223  -  INSERT INTO player_title (player_id, show_id) values (220,80113641)
224  -  INSERT INTO player_title (player_id, show_id) values (221,81054495)
225  -  INSERT INTO player_title (player_id, show_id) values (222,70212504)
226  -  INSERT INTO player_title (player_id, show_id) values (91,70212504)
227  -  INSERT INTO player_title (player_id, show_id) values (223,70212504)
228  -  INSERT INTO player_title (player_id, show_id) values (224,70212504)
229  -  INSERT INTO player_title (player_id, show_id) values (225,70212504)
230  -  INSERT INTO player_title (player_id, show_id) values (226,70212504)
231  -  INSERT INTO player_title (player_id, show_id) values (227,70212504)
232  -  INSERT INTO player_title (player_id, show_id) values (228,70212504)
233  -  INSER

329  -  INSERT INTO player_title (player_id, show_id) values (325,80060037)
330  -  INSERT INTO player_title (player_id, show_id) values (326,80060037)
331  -  INSERT INTO player_title (player_id, show_id) values (327,80060037)
332  -  INSERT INTO player_title (player_id, show_id) values (328,80060037)
333  -  INSERT INTO player_title (player_id, show_id) values (329,80060037)
334  -  INSERT INTO player_title (player_id, show_id) values (330,80060037)
335  -  INSERT INTO player_title (player_id, show_id) values (331,80060037)
336  -  INSERT INTO player_title (player_id, show_id) values (332,80060037)
337  -  INSERT INTO player_title (player_id, show_id) values (333,60003155)
338  -  INSERT INTO player_title (player_id, show_id) values (334,60003155)
339  -  INSERT INTO player_title (player_id, show_id) values (335,60003155)
340  -  INSERT INTO player_title (player_id, show_id) values (336,60003155)
341  -  INSERT INTO player_title (player_id, show_id) values (337,60003155)
342  -  INSE

437  -  INSERT INTO player_title (player_id, show_id) values (431,80208155)
438  -  INSERT INTO player_title (player_id, show_id) values (432,80208155)
439  -  INSERT INTO player_title (player_id, show_id) values (433,80208155)
440  -  INSERT INTO player_title (player_id, show_id) values (434,80208155)
441  -  INSERT INTO player_title (player_id, show_id) values (435,80208155)
442  -  INSERT INTO player_title (player_id, show_id) values (94,80208155)
443  -  INSERT INTO player_title (player_id, show_id) values (436,80208155)
444  -  INSERT INTO player_title (player_id, show_id) values (437,80208155)
445  -  INSERT INTO player_title (player_id, show_id) values (438,80208155)
446  -  INSERT INTO player_title (player_id, show_id) values (439,80208155)
447  -  INSERT INTO player_title (player_id, show_id) values (440,80208155)
448  -  INSERT INTO player_title (player_id, show_id) values (441,80162144)
449  -  INSERT INTO player_title (player_id, show_id) values (442,80162144)
450  -  INSER

...

23721  -  INSERT INTO player_title (player_id, show_id) values (13631,70300066)
23722  -  INSERT INTO player_title (player_id, show_id) values (4464,70300066)
23723  -  INSERT INTO player_title (player_id, show_id) values (11560,80027042)
23724  -  INSERT INTO player_title (player_id, show_id) values (15972,80027042)
23725  -  INSERT INTO player_title (player_id, show_id) values (11130,80027042)
23726  -  INSERT INTO player_title (player_id, show_id) values (15973,80027042)
23727  -  INSERT INTO player_title (player_id, show_id) values (15974,80027042)
23728  -  INSERT INTO player_title (player_id, show_id) values (15975,80027042)
23729  -  INSERT INTO player_title (player_id, show_id) values (15976,80027042)
23730  -  INSERT INTO player_title (player_id, show_id) values (15977,80027042)
23731  -  INSERT INTO player_title (player_id, show_id) values (7362,80027042)
23732  -  INSERT INTO player_title (player_id, show_id) values (15978,80027042)
23733  -  INSERT INTO player_title (player

23825  -  INSERT INTO player_title (player_id, show_id) values (16039,70281323)
23826  -  INSERT INTO player_title (player_id, show_id) values (1076,70281323)
23827  -  INSERT INTO player_title (player_id, show_id) values (3052,70281323)
23828  -  INSERT INTO player_title (player_id, show_id) values (16040,70281323)
23829  -  INSERT INTO player_title (player_id, show_id) values (16041,70281323)
23830  -  INSERT INTO player_title (player_id, show_id) values (16042,70281323)
23831  -  INSERT INTO player_title (player_id, show_id) values (7376,70281323)
23832  -  INSERT INTO player_title (player_id, show_id) values (16043,70281323)
23833  -  INSERT INTO player_title (player_id, show_id) values (16044,70281323)
23834  -  INSERT INTO player_title (player_id, show_id) values (14646,70153390)
23835  -  INSERT INTO player_title (player_id, show_id) values (4238,70153390)
23836  -  INSERT INTO player_title (player_id, show_id) values (3026,70153390)
23837  -  INSERT INTO player_title (player_id

23928  -  INSERT INTO player_title (player_id, show_id) values (16098,70153408)
23929  -  INSERT INTO player_title (player_id, show_id) values (16099,70153408)
23930  -  INSERT INTO player_title (player_id, show_id) values (16100,70153408)
23931  -  INSERT INTO player_title (player_id, show_id) values (16101,70153408)
23932  -  INSERT INTO player_title (player_id, show_id) values (7838,70153408)
23933  -  INSERT INTO player_title (player_id, show_id) values (16102,70153408)
23934  -  INSERT INTO player_title (player_id, show_id) values (16103,70153408)
23935  -  INSERT INTO player_title (player_id, show_id) values (16104,70153408)
23936  -  INSERT INTO player_title (player_id, show_id) values (16105,70153408)
23937  -  INSERT INTO player_title (player_id, show_id) values (16106,70153408)
23938  -  INSERT INTO player_title (player_id, show_id) values (16107,70305883)
23939  -  INSERT INTO player_title (player_id, show_id) values (13258,70305883)
23940  -  INSERT INTO player_title (playe

24031  -  INSERT INTO player_title (player_id, show_id) values (11519,70304245)
24032  -  INSERT INTO player_title (player_id, show_id) values (16171,70304245)
24033  -  INSERT INTO player_title (player_id, show_id) values (16172,70304245)
24034  -  INSERT INTO player_title (player_id, show_id) values (3539,80092654)
24035  -  INSERT INTO player_title (player_id, show_id) values (16173,80092654)
24036  -  INSERT INTO player_title (player_id, show_id) values (16174,80092654)
24037  -  INSERT INTO player_title (player_id, show_id) values (16175,80092654)
24038  -  INSERT INTO player_title (player_id, show_id) values (16176,80092654)
24039  -  INSERT INTO player_title (player_id, show_id) values (16177,80092654)
24040  -  INSERT INTO player_title (player_id, show_id) values (16178,80036407)
24041  -  INSERT INTO player_title (player_id, show_id) values (16179,80036407)
24042  -  INSERT INTO player_title (player_id, show_id) values (16180,80036407)
24043  -  INSERT INTO player_title (playe

24135  -  INSERT INTO player_title (player_id, show_id) values (16242,80121349)
24136  -  INSERT INTO player_title (player_id, show_id) values (10852,80121349)
24137  -  INSERT INTO player_title (player_id, show_id) values (16243,80121349)
24138  -  INSERT INTO player_title (player_id, show_id) values (16244,80121349)
24139  -  INSERT INTO player_title (player_id, show_id) values (5952,80095698)
24140  -  INSERT INTO player_title (player_id, show_id) values (7094,80095698)
24141  -  INSERT INTO player_title (player_id, show_id) values (7645,80095698)
24142  -  INSERT INTO player_title (player_id, show_id) values (16245,80095698)
24143  -  INSERT INTO player_title (player_id, show_id) values (16246,80095698)
24144  -  INSERT INTO player_title (player_id, show_id) values (16247,80095698)
24145  -  INSERT INTO player_title (player_id, show_id) values (10440,80095698)
24146  -  INSERT INTO player_title (player_id, show_id) values (8381,80095698)
24147  -  INSERT INTO player_title (player_i

In [33]:
for i in range(len(player_tile_statements)):
    conn.execute(player_tile_statements[i])

In [39]:
netflix_titles_countries = pd.DataFrame ({"Titles": df_new["show_id"],
                                          "Country": df_new["country"]}) 
netflix_titles_countries.Country = netflix_titles_countries.Country.str.split(', ')
netflix_titles_countries = netflix_titles_countries.explode('Country').reset_index(drop=True)
netflix_titles_countries.dropna(subset=['Country'], inplace=True)
netflix_titles_countries

,Titles,Country
0,81145628,United States
1,81145628,India
2,81145628,South Korea
3,81145628,China
4,80117401,United Kingdom
...,...,...
4329,70227189,Canada
4330,70272742,Canada
4331,70136122,United States
4332,80005756,Australia


In [40]:
netflix_titles_countries["Country"] = netflix_titles_countries["Country"].str.replace("'","''")

In [53]:
netflix_titles_countries.drop_duplicates (subset = ['Titles','Country'], inplace = True)
netflix_titles_countries["Country"] = netflix_titles_countries["Country"].str.replace(",","")
netflix_titles_countries.head (60)

,Titles,Country
0,81145628,United States
1,81145628,India
2,81145628,South Korea
3,81145628,China
4,80117401,United Kingdom
5,80163890,Spain
6,70304989,Bulgaria
7,70304989,United States
8,70304989,Spain
9,70304989,Canada


In [62]:
country_name_list = list(netflix_titles_countries['Country'])
show_id_list = list(netflix_titles_countries['Titles'])
country_title_statements = []
print (len(show_id_list))
for i in range(len(show_id_list)):
    print ("Country name: ",country_name_list[i], "-Show ID: ", show_id_list[i])
    statement_country_id = engine.execute(f"select country_id from country where country_name = '{country_name_list[i]}'").fetchall()
    print ("-Country ID: ",statement_country_id[0][0])
    country_title_statements.append (f"INSERT INTO title_country (country_id, show_id) values ({statement_country_id[0][0]},{show_id_list[i]})")
    print (i, " - " ,country_title_statements[i])

4197
Country name:  East Germany -Show ID:  60020949
-Country ID:  247
2680  -  INSERT INTO title_country (country_id, show_id) values (247,60020949)
Country name:  West Germany -Show ID:  60020949
-Country ID:  246
2681  -  INSERT INTO title_country (country_id, show_id) values (246,60020949)
Country name:  United States -Show ID:  60011597
-Country ID:  227
2682  -  INSERT INTO title_country (country_id, show_id) values (227,60011597)
Country name:  United States -Show ID:  70100379
-Country ID:  227
2683  -  INSERT INTO title_country (country_id, show_id) values (227,70100379)
Country name:  United Kingdom -Show ID:  70100379
-Country ID:  73
2684  -  INSERT INTO title_country (country_id, show_id) values (73,70100379)
Country name:  United States -Show ID:  70209245
-Country ID:  227
2685  -  INSERT INTO title_country (country_id, show_id) values (227,70209245)
Country name:  United States -Show ID:  70045863
-Country ID:  227
2686  -  INSERT INTO title_country (country_id, show_id

-Country ID:  102
2738  -  INSERT INTO title_country (country_id, show_id) values (102,80228146)
Country name:  United States -Show ID:  70169918
-Country ID:  227
2739  -  INSERT INTO title_country (country_id, show_id) values (227,70169918)
Country name:  India -Show ID:  80201810
-Country ID:  102
2740  -  INSERT INTO title_country (country_id, show_id) values (102,80201810)
Country name:  India -Show ID:  80221637
-Country ID:  102
2741  -  INSERT INTO title_country (country_id, show_id) values (102,80221637)
Country name:  France -Show ID:  80221637
-Country ID:  71
2742  -  INSERT INTO title_country (country_id, show_id) values (71,80221637)
Country name:  United States -Show ID:  80197301
-Country ID:  227
2743  -  INSERT INTO title_country (country_id, show_id) values (227,80197301)
Country name:  India -Show ID:  19207412
-Country ID:  102
2744  -  INSERT INTO title_country (country_id, show_id) values (102,19207412)
Country name:  United States -Show ID:  80217436
-Country ID

-Country ID:  227
2796  -  INSERT INTO title_country (country_id, show_id) values (227,80991400)
Country name:  United States -Show ID:  80231156
-Country ID:  227
2797  -  INSERT INTO title_country (country_id, show_id) values (227,80231156)
Country name:  United States -Show ID:  80191045
-Country ID:  227
2798  -  INSERT INTO title_country (country_id, show_id) values (227,80191045)
Country name:  Spain -Show ID:  81029383
-Country ID:  64
2799  -  INSERT INTO title_country (country_id, show_id) values (64,81029383)
Country name:  New Zealand -Show ID:  80178280
-Country ID:  167
2800  -  INSERT INTO title_country (country_id, show_id) values (167,80178280)
Country name:  United Kingdom -Show ID:  80178280
-Country ID:  73
2801  -  INSERT INTO title_country (country_id, show_id) values (73,80178280)
Country name:  Mexico -Show ID:  80158270
-Country ID:  153
2802  -  INSERT INTO title_country (country_id, show_id) values (153,80158270)
Country name:  France -Show ID:  70302836
-Coun

-Country ID:  98
2855  -  INSERT INTO title_country (country_id, show_id) values (98,80016401)
Country name:  Finland -Show ID:  80016401
-Country ID:  66
2856  -  INSERT INTO title_country (country_id, show_id) values (66,80016401)
Country name:  Norway -Show ID:  80016401
-Country ID:  163
2857  -  INSERT INTO title_country (country_id, show_id) values (163,80016401)
Country name:  United Kingdom -Show ID:  80016401
-Country ID:  73
2858  -  INSERT INTO title_country (country_id, show_id) values (73,80016401)
Country name:  Israel -Show ID:  80016401
-Country ID:  100
2859  -  INSERT INTO title_country (country_id, show_id) values (100,80016401)
Country name:  France -Show ID:  80016401
-Country ID:  71
2860  -  INSERT INTO title_country (country_id, show_id) values (71,80016401)
Country name:  United States -Show ID:  80016401
-Country ID:  227
2861  -  INSERT INTO title_country (country_id, show_id) values (227,80016401)
Country name:  Germany -Show ID:  80016401
-Country ID:  53
2

-Country ID:  227
2913  -  INSERT INTO title_country (country_id, show_id) values (227,80098490)
Country name:  United Kingdom -Show ID:  81024494
-Country ID:  73
2914  -  INSERT INTO title_country (country_id, show_id) values (73,81024494)
Country name:  United States -Show ID:  70176656
-Country ID:  227
2915  -  INSERT INTO title_country (country_id, show_id) values (227,70176656)
Country name:  United States -Show ID:  80223123
-Country ID:  227
2916  -  INSERT INTO title_country (country_id, show_id) values (227,80223123)
Country name:  United States -Show ID:  80214581
-Country ID:  227
2917  -  INSERT INTO title_country (country_id, show_id) values (227,80214581)
Country name:  South Korea -Show ID:  80211728
-Country ID:  119
2918  -  INSERT INTO title_country (country_id, show_id) values (119,80211728)
Country name:  Spain -Show ID:  80121386
-Country ID:  64
2919  -  INSERT INTO title_country (country_id, show_id) values (64,80121386)
Country name:  United States -Show ID:  

...

-Country ID:  227
3913  -  INSERT INTO title_country (country_id, show_id) values (227,80146759)
Country name:  Denmark -Show ID:  81073387
-Country ID:  55
3914  -  INSERT INTO title_country (country_id, show_id) values (55,81073387)
Country name:  Canada -Show ID:  80207606
-Country ID:  35
3915  -  INSERT INTO title_country (country_id, show_id) values (35,80207606)
Country name:  United States -Show ID:  80192186
-Country ID:  227
3916  -  INSERT INTO title_country (country_id, show_id) values (227,80192186)
Country name:  Canada -Show ID:  80216834
-Country ID:  35
3917  -  INSERT INTO title_country (country_id, show_id) values (35,80216834)
Country name:  United States -Show ID:  80216834
-Country ID:  227
3918  -  INSERT INTO title_country (country_id, show_id) values (227,80216834)
Country name:  United States -Show ID:  80174436
-Country ID:  227
3919  -  INSERT INTO title_country (country_id, show_id) values (227,80174436)
Country name:  United States -Show ID:  80174097
-Cou

-Country ID:  35
3971  -  INSERT INTO title_country (country_id, show_id) values (35,70142918)
Country name:  United Kingdom -Show ID:  80081790
-Country ID:  73
3972  -  INSERT INTO title_country (country_id, show_id) values (73,80081790)
Country name:  Mexico -Show ID:  80170690
-Country ID:  153
3973  -  INSERT INTO title_country (country_id, show_id) values (153,80170690)
Country name:  Italy -Show ID:  80160081
-Country ID:  107
3974  -  INSERT INTO title_country (country_id, show_id) values (107,80160081)
Country name:  Nigeria -Show ID:  80168301
-Country ID:  160
3975  -  INSERT INTO title_country (country_id, show_id) values (160,80168301)
Country name:  United States -Show ID:  80064273
-Country ID:  227
3976  -  INSERT INTO title_country (country_id, show_id) values (227,80064273)
Country name:  Belgium -Show ID:  80098475
-Country ID:  19
3977  -  INSERT INTO title_country (country_id, show_id) values (19,80098475)
Country name:  France -Show ID:  80098475
-Country ID:  71


-Country ID:  45
4029  -  INSERT INTO title_country (country_id, show_id) values (45,80103336)
Country name:  India -Show ID:  81066723
-Country ID:  102
4030  -  INSERT INTO title_country (country_id, show_id) values (102,81066723)
Country name:  India -Show ID:  80207445
-Country ID:  102
4031  -  INSERT INTO title_country (country_id, show_id) values (102,80207445)
Country name:  United States -Show ID:  80209751
-Country ID:  227
4032  -  INSERT INTO title_country (country_id, show_id) values (227,80209751)
Country name:  United States -Show ID:  80063773
-Country ID:  227
4033  -  INSERT INTO title_country (country_id, show_id) values (227,80063773)
Country name:  United Kingdom -Show ID:  81088284
-Country ID:  73
4034  -  INSERT INTO title_country (country_id, show_id) values (73,81088284)
Country name:  India -Show ID:  80987611
-Country ID:  102
4035  -  INSERT INTO title_country (country_id, show_id) values (102,80987611)
Country name:  United States -Show ID:  70060380
-Coun

-Country ID:  111
4089  -  INSERT INTO title_country (country_id, show_id) values (111,70205014)
Country name:  India -Show ID:  80122186
-Country ID:  102
4090  -  INSERT INTO title_country (country_id, show_id) values (102,80122186)
Country name:  India -Show ID:  80158479
-Country ID:  102
4091  -  INSERT INTO title_country (country_id, show_id) values (102,80158479)
Country name:  United States -Show ID:  80104318
-Country ID:  227
4092  -  INSERT INTO title_country (country_id, show_id) values (227,80104318)
Country name:  United Kingdom -Show ID:  70140438
-Country ID:  73
4093  -  INSERT INTO title_country (country_id, show_id) values (73,70140438)
Country name:  United States -Show ID:  80097468
-Country ID:  227
4094  -  INSERT INTO title_country (country_id, show_id) values (227,80097468)
Country name:  Argentina -Show ID:  80104237
-Country ID:  10
4095  -  INSERT INTO title_country (country_id, show_id) values (10,80104237)
Country name:  Spain -Show ID:  80093107
-Country 

-Country ID:  227
4147  -  INSERT INTO title_country (country_id, show_id) values (227,70281323)
Country name:  United States -Show ID:  70153390
-Country ID:  227
4148  -  INSERT INTO title_country (country_id, show_id) values (227,70153390)
Country name:  Canada -Show ID:  70153390
-Country ID:  35
4149  -  INSERT INTO title_country (country_id, show_id) values (35,70153390)
Country name:  United States -Show ID:  80178726
-Country ID:  227
4150  -  INSERT INTO title_country (country_id, show_id) values (227,80178726)
Country name:  United States -Show ID:  80109194
-Country ID:  227
4151  -  INSERT INTO title_country (country_id, show_id) values (227,80109194)
Country name:  Japan -Show ID:  80233218
-Country ID:  111
4152  -  INSERT INTO title_country (country_id, show_id) values (111,80233218)
Country name:  Japan -Show ID:  80196657
-Country ID:  111
4153  -  INSERT INTO title_country (country_id, show_id) values (111,80196657)
Country name:  India -Show ID:  80122239
-Country ID

In [63]:
for i in range(len(country_title_statements)):
    conn.execute(country_title_statements[i])

In [65]:
netflix_titles_listed_in = pd.DataFrame ({"Titles": df_new["show_id"],
                                          "Listed_In": df_new["listed_in"]}) 
netflix_titles_listed_in.Listed_In = netflix_titles_listed_in.Listed_In.str.split(', ')
netflix_titles_listed_in = netflix_titles_listed_in.explode('Listed_In').reset_index(drop=True)
netflix_titles_listed_in.dropna(subset=['Listed_In'], inplace=True)
netflix_titles_listed_in

,Titles,Listed_In
0,81145628,Children & Family Movies
1,81145628,Comedies
2,80117401,Stand-Up Comedy
3,80163890,Crime TV Shows
4,80163890,International TV Shows
...,...,...
7314,70136122,TV Dramas
7315,80005756,Kids' TV
7316,80005756,TV Comedies
7317,70153404,Classic & Cult TV


In [66]:
netflix_titles_listed_in["Listed_In"] = netflix_titles_listed_in["Listed_In"].str.replace("'","''")

In [69]:
netflix_titles_listed_in.drop_duplicates (subset = ['Titles','Listed_In'], inplace = True)
netflix_titles_listed_in["Listed_In"] = netflix_titles_listed_in["Listed_In"].str.replace(",","")
netflix_titles_listed_in

,Titles,Listed_In
0,81145628,Children & Family Movies
1,81145628,Comedies
2,80117401,Stand-Up Comedy
3,80163890,Crime TV Shows
4,80163890,International TV Shows
...,...,...
7314,70136122,TV Dramas
7315,80005756,Kids'' TV
7316,80005756,TV Comedies
7317,70153404,Classic & Cult TV


In [70]:
listed_in_list = list(netflix_titles_listed_in["Listed_In"])
show_id_list = list(netflix_titles_listed_in['Titles'])
listed_in_title_statements = []
print (len(show_id_list))
for i in range(len(show_id_list)):
    #print ("Country name: ",country_name_list[i], "-Show ID: ", show_id_list[i])
    statement_listed_in_id = engine.execute(f"select listed_in_id from listed_in where listed_in_name = '{listed_in_list[i]}'").fetchall()
    #print ("-Country ID: ",statement_country_id[0][0])
    listed_in_title_statements.append (f"INSERT INTO listed_in_title (listed_in_id, show_id) values ({statement_listed_in_id[0][0]},{show_id_list[i]})")
    print (i, " - " ,listed_in_title_statements[i])

7319
0  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (1,81145628)
1  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,81145628)
2  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (3,80117401)
3  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (4,80163890)
4  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (5,80163890)
5  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (6,80163890)
6  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,70304989)
7  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (8,70304989)
8  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (9,70304989)
9  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,70304990)
10  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (9,70304990)
11  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (3,80169755)
12  -  INSERT INTO listed_in_title (listed_in_id, show_i

105  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (17,80241248)
106  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (24,80241248)
107  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,81012295)
108  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (12,81012295)
109  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,81012295)
110  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80060037)
111  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (12,80060037)
112  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80060037)
113  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (1,60003155)
114  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,60003155)
115  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (25,60003155)
116  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,81177238)
117  -  INSERT INTO listed_in_ti

207  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (15,81147293)
208  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (12,81147293)
209  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,80991316)
210  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,80190588)
211  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (21,80190588)
212  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (3,70229037)
213  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (15,80037280)
214  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (12,80037280)
215  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,80058248)
216  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80058248)
217  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (15,70307661)
218  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (9,70307661)
219  -  INSERT INTO listed_in_

310  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,60000861)
311  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,1151375)
312  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,80187228)
313  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,81031008)
314  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (1,70099119)
315  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,70039185)
316  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (13,70039185)
317  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,81165769)
318  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,81165769)
319  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,70056433)
320  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,70113003)
321  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,60031232)
322  -  INSERT INTO listed_in_tit

412  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (16,70208520)
413  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (34,70208520)
414  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,80061314)
415  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,80158480)
416  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80158480)
417  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,80139506)
418  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (15,80139506)
419  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80205314)
420  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80205314)
421  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (33,80205314)
422  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,80158545)
423  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80158545)
424  -  INSERT INTO listed_in_tit

...

6816  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,81010166)
6817  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80112518)
6818  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (12,80112518)
6819  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,80202920)
6820  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (13,80202920)
6821  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (10,80215782)
6822  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80215782)
6823  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80215782)
6824  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,80169778)
6825  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,81035844)
6826  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,81035844)
6827  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,81035850)
6828  -  INSERT INTO 

6918  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80156683)
6919  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80156683)
6920  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (13,80156683)
6921  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,80141185)
6922  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80141185)
6923  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80141185)
6924  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (2,80126569)
6925  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,80171982)
6926  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80171982)
6927  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (9,80171982)
6928  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,80035467)
6929  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,81041391)
6930  -  INSERT INTO l

7019  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (21,60033223)
7020  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (7,80244992)
7021  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (21,80244992)
7022  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (9,80244992)
7023  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (4,80208373)
7024  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (5,80208373)
7025  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (23,80208373)
7026  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,80196139)
7027  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (33,80196139)
7028  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (14,80148180)
7029  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (11,607931)
7030  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (13,607931)
7031  -  INSERT INTO lis

7121  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (17,70281312)
7122  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (36,70281312)
7123  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (17,70295760)
7124  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (31,70295760)
7125  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (18,70295760)
7126  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (4,80119379)
7127  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (5,80119379)
7128  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (27,80119379)
7129  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (23,80171611)
7130  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (23,80077977)
7131  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (4,80053653)
7132  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (31,80053653)
7133  -  INSERT INT

7223  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (5,80233218)
7224  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (17,80233218)
7225  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (36,80233218)
7226  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (30,80196657)
7227  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (5,80196657)
7228  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (24,80196657)
7229  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (5,80122239)
7230  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (20,80122239)
7231  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (17,80122239)
7232  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (31,80057281)
7233  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (18,80057281)
7234  -  INSERT INTO listed_in_title (listed_in_id, show_id) values (38,80057281)
7235  -  INSERT INT

In [71]:
for i in range(len(listed_in_title_statements)):
    conn.execute(listed_in_title_statements[i])

In [75]:
netflix_titles_directors = pd.DataFrame ({"Titles": df_new["show_id"],
                                          "directors": df_new["director"]}) 
netflix_titles_directors.directors = netflix_titles_directors.directors.str.split(', ')
netflix_titles_directors = netflix_titles_directors.explode('directors').reset_index(drop=True)
netflix_titles_directors.dropna(subset=['directors'], inplace=True)
netflix_titles_directors.head()

,Titles,directors
0,81145628,Richard Finn
1,81145628,Tim Maltby
4,70304989,Gabe Ibáñez
5,70304990,Henrik Ruben Genz
6,80169755,José Miguel Contreras


In [76]:
netflix_titles_directors["directors"] = netflix_titles_directors["directors"].str.replace("'","''")

In [77]:
netflix_titles_directors.drop_duplicates (subset = ['Titles','directors'], inplace = True)
netflix_titles_directors["directors"] = netflix_titles_directors["directors"].str.replace(",","")
netflix_titles_directors

,Titles,directors
0,81145628,Richard Finn
1,81145628,Tim Maltby
4,70304989,Gabe Ibáñez
5,70304990,Henrik Ruben Genz
6,80169755,José Miguel Contreras
...,...,...
3735,80085439,G.J. Echternkamp
3736,80011846,Zatella Beatty
3737,80064521,Vlad Yudin
3766,80027042,Glen Winter


In [85]:
directors_list = list(netflix_titles_directors["directors"])
show_id_list = list(netflix_titles_directors['Titles'])
directors_title_statements = []
print (len(show_id_list))
for i in range(len(show_id_list)):
    print ("Director name: ",directors_list[i], "-Show ID: ", show_id_list[i])
    statement_directors_id = engine.execute(f"select director_id from director where director_name = '{directors_list[i]}'").fetchall()
    print ("-Director ID: ",statement_directors_id[0][0])
    directors_title_statements.append (f"INSERT INTO director_title (director_id, show_id) values ({statement_directors_id[0][0]},{show_id_list[i]})")
    print (i, " - " ,directors_title_statements[i])

3498
Director name:  Richard Finn -Show ID:  81145628
-Director ID:  3172
0  -  INSERT INTO director_title (director_id, show_id) values (3172,81145628)
Director name:  Tim Maltby -Show ID:  81145628
-Director ID:  3173
1  -  INSERT INTO director_title (director_id, show_id) values (3173,81145628)
Director name:  Gabe Ibáñez -Show ID:  70304989
-Director ID:  3174
2  -  INSERT INTO director_title (director_id, show_id) values (3174,70304989)
Director name:  Henrik Ruben Genz -Show ID:  70304990
-Director ID:  3175
3  -  INSERT INTO director_title (director_id, show_id) values (3175,70304990)
Director name:  José Miguel Contreras -Show ID:  80169755
-Director ID:  3176
4  -  INSERT INTO director_title (director_id, show_id) values (3176,80169755)
Director name:  Daniel Alfredson -Show ID:  70299204
-Director ID:  3177
5  -  INSERT INTO director_title (director_id, show_id) values (3177,70299204)
Director name:  Tom O''Brien -Show ID:  80060297
-Director ID:  3178
6  -  INSERT INTO direc

-Director ID:  3227
55  -  INSERT INTO director_title (director_id, show_id) values (3227,81171121)
Director name:  Michael Arias -Show ID:  81171121
-Director ID:  3228
56  -  INSERT INTO director_title (director_id, show_id) values (3228,81171121)
Director name:  Masaru Matsumoto -Show ID:  81171121
-Director ID:  3229
57  -  INSERT INTO director_title (director_id, show_id) values (3229,81171121)
Director name:  Arthell Isom -Show ID:  81171121
-Director ID:  3230
58  -  INSERT INTO director_title (director_id, show_id) values (3230,81171121)
Director name:  Henry Thurlow -Show ID:  81171121
-Director ID:  3231
59  -  INSERT INTO director_title (director_id, show_id) values (3231,81171121)
Director name:  Takanobu Mizuno -Show ID:  81171121
-Director ID:  3232
60  -  INSERT INTO director_title (director_id, show_id) values (3232,81171121)
Director name:  Elsa Nakamichi -Show ID:  81171121
-Director ID:  3233
61  -  INSERT INTO director_title (director_id, show_id) values (3233,81171

-Director ID:  3283
111  -  INSERT INTO director_title (director_id, show_id) values (3283,80987075)
Director name:  Morgan Neville -Show ID:  80066798
-Director ID:  3284
112  -  INSERT INTO director_title (director_id, show_id) values (3284,80066798)
Director name:  Teo Konuralp -Show ID:  81147293
-Director ID:  3285
113  -  INSERT INTO director_title (director_id, show_id) values (3285,81147293)
Director name:  Jesse V. Johnson -Show ID:  80991316
-Director ID:  3211
114  -  INSERT INTO director_title (director_id, show_id) values (3211,80991316)
Director name:  Chris Perkel -Show ID:  80190588
-Director ID:  3286
115  -  INSERT INTO director_title (director_id, show_id) values (3286,80190588)
Director name:  Manny Rodriguez -Show ID:  70229037
-Director ID:  3287
116  -  INSERT INTO director_title (director_id, show_id) values (3287,70229037)
Director name:  Robert Eggers -Show ID:  80037280
-Director ID:  3288
117  -  INSERT INTO director_title (director_id, show_id) values (3288

-Director ID:  3336
167  -  INSERT INTO director_title (director_id, show_id) values (3336,80991570)
Director name:  Trent Haaga -Show ID:  80184129
-Director ID:  3337
168  -  INSERT INTO director_title (director_id, show_id) values (3337,80184129)
Director name:  Mary Harron -Show ID:  60000861
-Director ID:  3338
169  -  INSERT INTO director_title (director_id, show_id) values (3338,60000861)
Director name:  Roger Donaldson -Show ID:  1151375
-Director ID:  3339
170  -  INSERT INTO director_title (director_id, show_id) values (3339,1151375)
Director name:  Steven C. Miller -Show ID:  80187228
-Director ID:  3340
171  -  INSERT INTO director_title (director_id, show_id) values (3340,80187228)
Director name:  Richard Miron -Show ID:  81031008
-Director ID:  3341
172  -  INSERT INTO director_title (director_id, show_id) values (3341,81031008)
Director name:  Tony Leondis -Show ID:  70099119
-Director ID:  3342
173  -  INSERT INTO director_title (director_id, show_id) values (3342,70099

-Director ID:  3385
222  -  INSERT INTO director_title (director_id, show_id) values (3385,80184055)
Director name:  Wynn Padula -Show ID:  80184055
-Director ID:  3386
223  -  INSERT INTO director_title (director_id, show_id) values (3386,80184055)
Director name:  Thierry Demaizière -Show ID:  80160358
-Director ID:  3387
224  -  INSERT INTO director_title (director_id, show_id) values (3387,80160358)
Director name:  Alban Teurlai -Show ID:  80160358
-Director ID:  3388
225  -  INSERT INTO director_title (director_id, show_id) values (3388,80160358)
Director name:  Latif Faiziyev -Show ID:  80157134
-Director ID:  3389
226  -  INSERT INTO director_title (director_id, show_id) values (3389,80157134)
Director name:  Umesh Mehra -Show ID:  80157134
-Director ID:  3383
227  -  INSERT INTO director_title (director_id, show_id) values (3383,80157134)
Director name:  Gastón Duprat -Show ID:  80141891
-Director ID:  3391
228  -  INSERT INTO director_title (director_id, show_id) values (3391,8

...

-Director ID:  5827
3245  -  INSERT INTO director_title (director_id, show_id) values (5827,70222860)
Director name:  Steve Carr -Show ID:  70222860
-Director ID:  4337
3246  -  INSERT INTO director_title (director_id, show_id) values (4337,70222860)
Director name:  Griffin Dunne -Show ID:  70222860
-Director ID:  3455
3247  -  INSERT INTO director_title (director_id, show_id) values (3455,70222860)
Director name:  Steve Brill -Show ID:  70222860
-Director ID:  5830
3248  -  INSERT INTO director_title (director_id, show_id) values (5830,70222860)
Director name:  James Duffy -Show ID:  70222860
-Director ID:  5831
3249  -  INSERT INTO director_title (director_id, show_id) values (5831,70222860)
Director name:  Jonathan van Tulleken -Show ID:  70222860
-Director ID:  5832
3250  -  INSERT INTO director_title (director_id, show_id) values (5832,70222860)
Director name:  Elizabeth Banks -Show ID:  70222860
-Director ID:  5833
3251  -  INSERT INTO director_title (director_id, show_id) values

-Director ID:  5874
3299  -  INSERT INTO director_title (director_id, show_id) values (5874,80164904)
Director name:  Kranti Kanade -Show ID:  80157889
-Director ID:  5875
3300  -  INSERT INTO director_title (director_id, show_id) values (5875,80157889)
Director name:  Dar Gai -Show ID:  81094082
-Director ID:  5876
3301  -  INSERT INTO director_title (director_id, show_id) values (5876,81094082)
Director name:  Ricky Gervais -Show ID:  80048940
-Director ID:  5104
3302  -  INSERT INTO director_title (director_id, show_id) values (5104,80048940)
Director name:  Jon Greenhalgh -Show ID:  80044093
-Director ID:  5877
3303  -  INSERT INTO director_title (director_id, show_id) values (5877,80044093)
Director name:  Christopher S. Rech -Show ID:  80044562
-Director ID:  5878
3304  -  INSERT INTO director_title (director_id, show_id) values (5878,80044562)
Director name:  Brandon Kimber -Show ID:  80044562
-Director ID:  5879
3305  -  INSERT INTO director_title (director_id, show_id) values 

-Director ID:  5916
3353  -  INSERT INTO director_title (director_id, show_id) values (5916,80213712)
Director name:  Rebecca Addelman -Show ID:  80207606
-Director ID:  5917
3354  -  INSERT INTO director_title (director_id, show_id) values (5917,80207606)
Director name:  Olivia Milch -Show ID:  80192186
-Director ID:  5918
3355  -  INSERT INTO director_title (director_id, show_id) values (5918,80192186)
Director name:  Mark Raso -Show ID:  80216834
-Director ID:  5919
3356  -  INSERT INTO director_title (director_id, show_id) values (5919,80216834)
Director name:  David Sington -Show ID:  80174436
-Director ID:  4671
3357  -  INSERT INTO director_title (director_id, show_id) values (4671,80174436)
Director name:  Heather Walsh -Show ID:  80174436
-Director ID:  5920
3358  -  INSERT INTO director_title (director_id, show_id) values (5920,80174436)
Director name:  Chris Stokes -Show ID:  80174097
-Director ID:  4765
3359  -  INSERT INTO director_title (director_id, show_id) values (4765

-Director ID:  5675
3408  -  INSERT INTO director_title (director_id, show_id) values (5675,80058247)
Director name:  Josh Alexander -Show ID:  80058247
-Director ID:  5953
3409  -  INSERT INTO director_title (director_id, show_id) values (5953,80058247)
Director name:  Greg Young -Show ID:  80058247
-Director ID:  5954
3410  -  INSERT INTO director_title (director_id, show_id) values (5954,80058247)
Director name:  Werner Herzog -Show ID:  70302184
-Director ID:  3449
3411  -  INSERT INTO director_title (director_id, show_id) values (3449,70302184)
Director name:  Ken Barbet -Show ID:  81060043
-Director ID:  5955
3412  -  INSERT INTO director_title (director_id, show_id) values (5955,81060043)
Director name:  Sam Taylor-Johnson -Show ID:  70124562
-Director ID:  5956
3413  -  INSERT INTO director_title (director_id, show_id) values (5956,70124562)
Director name:  Marie Madinier -Show ID:  80156683
-Director ID:  5957
3414  -  INSERT INTO director_title (director_id, show_id) values (

-Director ID:  3407
3463  -  INSERT INTO director_title (director_id, show_id) values (3407,70298251)
Director name:  Kundan Shah -Show ID:  60002111
-Director ID:  4166
3464  -  INSERT INTO director_title (director_id, show_id) values (4166,60002111)
Director name:  Farah Khan -Show ID:  70000196
-Director ID:  5337
3465  -  INSERT INTO director_title (director_id, show_id) values (5337,70000196)
Director name:  Tathagata Banerjee -Show ID:  80245519
-Director ID:  5994
3466  -  INSERT INTO director_title (director_id, show_id) values (5994,80245519)
Director name:  Ananda Krishnan -Show ID:  80245329
-Director ID:  5995
3467  -  INSERT INTO director_title (director_id, show_id) values (5995,80245329)
Director name:  Farah Khan -Show ID:  70083535
-Director ID:  5337
3468  -  INSERT INTO director_title (director_id, show_id) values (5337,70083535)
Director name:  Aziz Mirza -Show ID:  60000722
-Director ID:  3671
3469  -  INSERT INTO director_title (director_id, show_id) values (3671,

In [ ]:
for i in range(len(directors_title_statements)):
    conn.execute(directors_title_statements[i])

In [ ]:
columns = inspector.get_columns('title')
columns

In [ ]:
for cols in columns: 
    print(cols['name'], cols['type'])

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Title = Base.classes.title

In [ ]:
session = Session(engine)

In [ ]:
results = session.query(Title.show_id, Title.award, Title.movie_id).order_by(Title.movie_id.desc()).all()

In [ ]:
print(results)

In [ ]:
results_2 = session.query(Title.director).all()
print(results_2)
#Director_df = pd.DataFrame(results_2,columns=['director'])
#prcp_df = prcp_df.set_index("director")
#Director_df.head()

In [ ]:
Director_df = pd.DataFrame(results_2,columns=['director'])
#Director_df = Director_df.set_index("director")
Director_df = Director_df.reset_index(drop=True)
Director_df.head()

In [ ]:
from sqlalchemy import insert

In [ ]:
for cols in results_2: 
    print(cols[0])

In [ ]:
engine.execute("Select * from Director").fetchall()

In [ ]:
Director_df.director = Director_df.director.str.split(',')
Director_df = Director_df.explode('director').reset_index(drop=True)
Director_df.drop_duplicates(subset=['director'], inplace=True)
Director_df.dropna(subset=['director'],inplace=True)
#Director_df = Director_df[Director_df.director != "None"]
Director_df

In [ ]:
Director_df["director"] = Director_df["director"].str.replace("'","''")

In [ ]:
for cols in Director_df['director']: 
    statement = f"INSERT INTO Director (director_name) VALUES ('{cols}')"
    conn.execute(statement)

In [ ]:
#Country
results_3 = session.query(Title.country).all()
print(results_3)

In [ ]:
#Country
Country_df = pd.DataFrame(results_3,columns=['country'])
Country_df = Country_df.reset_index(drop=True)
Country_df.head()

In [ ]:
Country_df.country = Country_df.country.str.split(',')
Country_df = Country_df.explode('country').reset_index(drop=True)
Country_df.drop_duplicates(subset=['country'], inplace=True)
Country_df.dropna(subset=['country'],inplace=True)
#Director_df = Director_df[Director_df.director != "None"]
Country_df

In [ ]:
for h in Country_df['country']: 
    statement_2 = f"INSERT INTO country (country_name) VALUES ('{h}')"
    conn.execute(statement_2)
    #print(col)

In [ ]:
#Rating
results_4 = session.query(Title.rating).all()
print(results_4)

In [ ]:
pg_rating_df = pd.DataFrame(results_4,columns=['rating'])
pg_rating_df = pg_rating_df.reset_index(drop=True)
pg_rating_df.head()

In [ ]:
for cols in results_4:
    print(cols[0])

In [ ]:
engine.execute("Select * from pg_rating").fetchall()

In [ ]:
pg_rating_df.drop_duplicates(subset=['rating'], inplace=True)
pg_rating_df.dropna(subset=['rating'],inplace=True)
pg_rating_df

In [ ]:
for cols in pg_rating_df['rating']:
    statement = f"INSERT INTO pg_rating (pg_rating_name) VALUES ('{cols}')"
    conn.execute(statement)

In [ ]:
######### New Code #######
netflix_titles_path = "TMDB_OMDB.csv"

# Read the mouse data and the study results
netflix_titles = pd.read_csv(netflix_titles_path)
netflix_titles.replace('', np.nan, inplace=True) 
netflix_titles.drop_duplicates(subset=['show_id'], inplace=True)
#netflix_titles.dropna(subset=['Movie ID','director', 'cast', 'country', 'listed_in'], inplace=True)
#netflix_titles = netflix_titles_temp.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

netflix_titles

In [ ]:
netflix_titles_directors = pd.DataFrame ({"Titles": netflix_titles["show_id"],
                                          "directors": netflix_titles["director"]}) 
netflix_titles_directors.directors = netflix_titles_directors.directors.str.split(', ')
netflix_titles_directors = netflix_titles_directors.explode('directors').reset_index(drop=True)
netflix_titles_directors.dropna(subset=['directors'], inplace=True)
netflix_titles_directors.head(30)